Notebook a utilizar el Google Colab para el entrenamiento de YOLO

In [ ]:
!nvidia-smi

# Deberías ver algo como:
# | NVIDIA-SMI 525.xx.xx    Driver Version: 525.xx.xx    CUDA Version: 12.x |
# |   0  Tesla T4           ...

import torch
print(f"\n✅ CUDA disponible: {torch.cuda.is_available()}")
print(f"✅ GPU detectada: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else '❌ NO GPU'}")

# Si NO ves la GPU:
# 1. Ve a: Entorno de ejecución > Cambiar tipo de entorno de ejecución
# 2. Selecciona: Acelerador por hardware > GPU
# 3. Guarda y ejecuta esta celda de nuevo

In [ ]:
print("📦 Instalando Ultralytics...")
!pip install ultralytics -q

# Verificar instalación
from ultralytics import YOLO
import ultralytics
print(f"✅ Ultralytics instalado: versión {ultralytics.__version__}")

In [ ]:
from google.colab import drive
print("🔗 Montando Google Drive...")
drive.mount('/content/drive')

# Te pedirá permisos:
# 1. Clic en el enlace
# 2. Selecciona tu cuenta de Google
# 3. Permite el acceso
# 4. Copia el código y pégalo

print("✅ Drive montado correctamente")

# Crear carpeta del proyecto
import os
project_path = '/content/drive/MyDrive/YOLO11_Dron'
os.makedirs(project_path, exist_ok=True)
print(f"📁 Carpeta del proyecto: {project_path}")

In [ ]:
print("🔍 Verificando configuración...\n")

# 1. GPU
if torch.cuda.is_available():
    print("✅ GPU: OK")
    print(f"   └─ {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU: NO DETECTADA")
    print("   └─ Ve a: Entorno de ejecución > Cambiar tipo > GPU")

# 2. Drive
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Google Drive: OK")
else:
    print("❌ Google Drive: NO MONTADO")

# 3. Ultralytics
try:
    from ultralytics import YOLO
    print(f"✅ Ultralytics: OK (v{ultralytics.__version__})")
except:
    print("❌ Ultralytics: NO INSTALADO")

print("\n🎉 Si todo está en ✅, ¡estás listo para entrenar!")

In [ ]:
from ultralytics import YOLO

# Cargar modelo
model = YOLO('yolo11m.pt')  # Medium = mejor para VisDrone

# Entrenamiento completo
results = model.train(
    data='VisDrone.yaml',
    epochs=100,                      # Entrenamiento completo
    imgsz=640,
    batch=16,                        # Reducir a 8 si da error
    device=0,

    # Guardar en Drive
    project='/content/drive/MyDrive/YOLO11_Dron/runs',
    name='visdrone_completo',

    # Checkpoints frecuentes por si se corta
    save=True,
    save_period=10,                  # Guardar cada 10 épocas

    # Configuración óptima
    patience=50,
    optimizer='AdamW',
    lr0=0.001,

    # Augmentaciones
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,

    verbose=True
)

print("\n🎉 ¡ENTRENAMIENTO COMPLETADO!")

In [ ]:
from ultralytics import YOLO

print("🔄 Reanudando entrenamiento...")

# Cargar último checkpoint
model = YOLO('/content/drive/MyDrive/YOLO11_Dron/runs/visdrone_completo/weights/last.pt')

# Continuar entrenamiento
results = model.train(
    resume=True,  # ← IMPORTANTE
    project='/content/drive/MyDrive/YOLO11_Dron/runs',
    name='visdrone_completo'
)

print("✅ Entrenamiento reanudado")

In [ ]:
from IPython.display import Image, display
import os

results_path = '/content/drive/MyDrive/YOLO11_Dron/runs/visdrone_completo'

print("📊 Visualizando resultados...\n")

# Mostrar curvas de entrenamiento
if os.path.exists(f'{results_path}/results.png'):
    print("📈 Curvas de entrenamiento:")
    display(Image(filename=f'{results_path}/results.png', width=800))

# Mostrar matriz de confusión
if os.path.exists(f'{results_path}/confusion_matrix.png'):
    print("\n🎯 Matriz de confusión:")
    display(Image(filename=f'{results_path}/confusion_matrix.png', width=600))

# Mostrar ejemplos de predicciones
if os.path.exists(f'{results_path}/val_batch0_pred.jpg'):
    print("\n🖼️ Ejemplos de predicciones:")
    display(Image(filename=f'{results_path}/val_batch0_pred.jpg', width=800))

# Ver métricas
import pandas as pd
if os.path.exists(f'{results_path}/results.csv'):
    df = pd.read_csv(f'{results_path}/results.csv')
    print("\n📊 Métricas finales:")
    print(df[['epoch', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']].tail())

In [ ]:
from ultralytics import YOLO

# Cargar mejor modelo
best_model = YOLO('/content/drive/MyDrive/YOLO11_Dron/runs/visdrone_completo/weights/best.pt')

# Validar
print("🎯 Validando modelo...")
metrics = best_model.val()

print(f"\n📊 RESULTADOS FINALES:")
print(f"   mAP50: {metrics.box.map50:.4f} ({metrics.box.map50*100:.1f}%)")
print(f"   mAP50-95: {metrics.box.map:.4f} ({metrics.box.map*100:.1f}%)")
print(f"   Precision: {metrics.box.mp:.4f}")
print(f"   Recall: {metrics.box.mr:.4f}")

In [ ]:
print("📦 Comprimiendo modelo para descargar...")

# Comprimir carpeta de pesos
!cd /content/drive/MyDrive/YOLO11_Dron/runs/visdrone_completo && \
 zip -r modelo_final.zip weights/

print("✅ Modelo comprimido")

# Descargar
from google.colab import files
files.download('/content/drive/MyDrive/YOLO11_Dron/runs/visdrone_completo/modelo_final.zip')

print("\n🎉 ¡Listo! El archivo se está descargando a tu PC")
print("\nContenido del ZIP:")
print("  ├─ best.pt     ← Mejor modelo (usar este)")
print("  └─ last.pt     ← Último checkpoint")